In [65]:
'''
Do labeling only for mRNA puncta. 

if imaged proteins are not looking like puncta, then labeling of these proteins more likely does not make sense
'''



# creates a list of paths (path_list) with working files

import os

# Create a list of paths the name of which has two variables: mNRA_Name and SubSET
# def get_path(mNRA_Red, mRNA_Green, SubSET):

def get_path():


    input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'

    path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                
                # Read the file content and convert it to lowercase
                
                path_list.append(root)

    # Return the list of paths
    return path_list


In [66]:
# runs function get_path and saves path list as .txt file

path_list = get_path()

with open('path_list.txt', 'w') as file:
    for item in path_list:
        file.write(item + '\n')
        
len(path_list)

39

In [68]:
# function for parallelization that takes masked and raw images of mRNA from the path list and labels puncta


from concurrent.futures import ThreadPoolExecutor
from skimage import io
import os
import re
import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.filters import gaussian
from skimage.measure import label
import tifffile

# path_list = [fr'\\?\D:\Projects\LocProt\Hi-Plex\Leica\Cell1\aligned\ML_test\Analysis']

def process_image(path):
    with open(os.path.join(path, os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()

    # Extract channel numbers and names using regular expressions
    # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+)', content)[0]))
    # channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+)', content)[0]
    
    channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+),(\d+)', content)[0])) #ONLY for 3rd channel
    channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+),(\w+)', content)[0] #ONLY for 3rd channel      

    # Combine channel numbers and names into a dictionary
    channel_info = dict(zip(channel_numbers, channel_names))

    mask0 = channel_names[0]
    raw0 = channel_numbers[0]
    mask1 = channel_names[1]
    raw1 = channel_numbers[1]
    mask2 = channel_names[2]
    raw2 = channel_numbers[2]
    mask3 = channel_names[3]
    raw3 = channel_numbers[3]
        
    mRNA1_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask1}_filtered_image.tif')
    mRNA2_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask2}_filtered_image.tif') 
    mRNA3_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask3}_filtered_image.tif')        

    Cytosol_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Cytosol_image.tif')
    raw1_image_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw1}.tif')
    raw2_image_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw2}.tif')
    raw3_image_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw3}.tif')

    full_mask_image1 = tifffile.imread(mRNA1_Name_SubSET_mask_path)
    full_mask_image2 = tifffile.imread(mRNA2_Name_SubSET_mask_path)
    full_mask_image3 = tifffile.imread(mRNA3_Name_SubSET_mask_path)

    full_raw_image1 = tifffile.imread(raw1_image_path)
    full_raw_image2 = tifffile.imread(raw2_image_path)
    full_raw_image3 = tifffile.imread(raw3_image_path)
    cytosol = tifffile.imread(Cytosol_SubSET_mask_path)

    # threshold_value_smoothed = filters.threshold_otsu(binary_mask)
    threshold_value_mask1 = np.max(full_mask_image1)/8
    threshold_value_mask2 = np.max(full_mask_image2)/8
    threshold_value_mask3 = np.max(full_mask_image3)/8

    smoothed_mask_image1 = full_mask_image1 > threshold_value_mask1  
    smoothed_mask_image2 = full_mask_image2 > threshold_value_mask2  
    smoothed_mask_image3 = full_mask_image3 > threshold_value_mask3  
 
    mask_image1 = cytosol * smoothed_mask_image1
    mask_image2 = cytosol * smoothed_mask_image2
    mask_image3 = cytosol * smoothed_mask_image3
    raw_image1 = cytosol * full_raw_image1
    raw_image2 = cytosol * full_raw_image2
    raw_image3 = cytosol * full_raw_image3
    
    # Apply a 3D Gaussian filter to the raw image
    smoothed_image1 = gaussian(raw_image1, sigma=3)
    smoothed_image2 = gaussian(raw_image2, sigma=3)
    smoothed_image3 = gaussian(raw_image3, sigma=3)

    # Threshold the smoothed image
    threshold_value1 = 0.1 * np.max(smoothed_image1)
    threshold_value2 = 0.1 * np.max(smoothed_image2)
    threshold_value3 = 0.1 * np.max(smoothed_image3)

    thresholded_image1 = (smoothed_image1 > threshold_value1)
    thresholded_image2 = (smoothed_image2 > threshold_value2)
    thresholded_image3 = (smoothed_image3 > threshold_value3)

    # Mask out any pixels that are not part of a signal
    masked_image1 = thresholded_image1 * mask_image1
    masked_image2 = thresholded_image2 * mask_image2
    masked_image3 = thresholded_image3 * mask_image3

    # Compute the distance transform
    distance1 = ndi.distance_transform_edt(masked_image1)
    distance2 = ndi.distance_transform_edt(masked_image2)
    distance3 = ndi.distance_transform_edt(masked_image3)

    # Find the local maxima in the distance map
    local_maxi1 = ndi.maximum_filter(distance1, size=10) == distance1
    local_maxi2 = ndi.maximum_filter(distance2, size=10) == distance2
    local_maxi3 = ndi.maximum_filter(distance3, size=10) == distance3

    # Label each local maximum with a unique label
    markers1 = label(local_maxi1)
    markers2 = label(local_maxi2)
    markers3 = label(local_maxi3)

    # Apply the watershed algorithm to segment the signals
    labels1 = watershed(-distance1, markers1, mask=masked_image1)
    labels2 = watershed(-distance2, markers2, mask=masked_image2)
    labels3 = watershed(-distance3, markers3, mask=masked_image3)

    # Label each segment with a unique label
    labeled_image1 = label(labels1)
    labeled_image2 = label(labels2)
    labeled_image3 = label(labels3)

    directory_name = 'Segmented'
    segmented_dir_path = os.path.join(path, directory_name)

    # Add these lines
    if not os.path.exists(segmented_dir_path):
        os.makedirs(segmented_dir_path)

    #saves the segmented images in the same directory the processed images are
    Labeled_mrna1_path = os.path.join(segmented_dir_path, f'Labeled_{mask1}.tif')
    Labeled_mrna2_path = os.path.join(segmented_dir_path, f'Labeled_{mask2}.tif')
    Labeled_mrna3_path = os.path.join(segmented_dir_path, f'Labeled_{mask3}.tif')

    io.imsave(Labeled_mrna1_path, labeled_image1.astype('uint16'))
    io.imsave(Labeled_mrna2_path, labeled_image2.astype('uint16'))
    io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))

    print(f"Labeled image 1 saved in: {os.path.abspath(Labeled_mrna1_path)}")
    print(f"Labeled image 2 saved in: {os.path.abspath(Labeled_mrna2_path)}")
    print(f"Labeled image 3 saved in: {os.path.abspath(Labeled_mrna3_path)}")

    return

In [69]:
# function of parallelization of process_image() function

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def label_puncta_parallel(path_list):
    with ThreadPoolExecutor(max_workers=8) as executor: # specify the number of cpu cores you want to use
        futures = {executor.submit(process_image, path): path for path in path_list}

        for future in tqdm(as_completed(futures), total=len(path_list), desc="Processing Images"):
            pass


In [2]:
path_list

In [1]:
label_puncta_parallel(path_list)